In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
from minepy import MINE
from mlxtend.regressor import StackingRegressor

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
def evalerror(y, y_pred):
    loss = np.sum(np.square(y - y_pred))
    n = len(y)
    return loss / n

In [21]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('train/test.csv')

In [22]:
y = pd.read_csv('train/y.csv')

In [4]:
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.3
param['min_child_weight'] = 2

num_round = 10000


In [23]:
clf_gt2 = ensemble.GradientBoostingRegressor(max_depth=1, n_estimators=190, random_state=1)
clf_gt2.fit(train, y)

# Set a minimum threshold of 0.25 
model1 = SelectFromModel(clf_gt2, prefit=True) 
train = pd.DataFrame(model1.transform(train))
test = pd.DataFrame(model1.transform(test))

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_boost_round=4200, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0]) 

0.0244562


Stacking 线上

In [48]:
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, max_depth=2, subsample=0.8, learning_rate=0.01, random_state=1, max_features=0.3)
modle0  = xgb.XGBRegressor(learning_rate=0.01, max_depth=3, colsample_bytree=0.3, subsample=0.8, seed=0, n_estimators=1100)
modle1  = xgb.XGBRegressor(learning_rate=0.01, max_depth=4, colsample_bytree=0.3, subsample=0.8, seed=0, n_estimators=1200)
sv = svm.SVR()
tree1 = tree.ExtraTreeRegressor()
xgb1 = xgb.XGBRegressor()
lgb1 = lgb.LGBMRegressor()
et = linear_model.LinearRegression()

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3, 
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=250, 
                         seed=1)

base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['gb', lgb1],
              ['gb', xgb1],
              ['lgb', clf1],]

folds = list(KFold(len(train), n_folds=5, random_state=0))
S_train = np.zeros((train.shape[0], len(base_model)))
S_test = np.zeros((test.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((test.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = train.ix[train_idx, :]
        X_valid = train.ix[test_idx, :]
        Y = y.ix[train_idx, :]
        clf.fit(X_train, Y['Y'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(test)       
    S_test[:, index] = S_test_i.mean(1)
    
print('Done')

the model 0
the model 1
the model 2
the model 3
the model 4
the model 5
Done


In [49]:
linreg = svm.SVR()
linreg1 = linear_model.LinearRegression()
linreg1.fit(S_train, y)

result = linreg1.predict(S_test)
evalerror(pd.DataFrame(result)[0],  y_valid['Y'])

0.046595632236128122

In [50]:
sub = pd.read_csv('data/测试A-答案模板.csv', names=['ID'])
sub['res'] = pd.DataFrame(result)[0]
sub

,ID,res
0,ID716,2.694028
1,ID717,2.717678
2,ID719,2.899774
3,ID720,2.960420
4,ID721,2.790239
5,ID722,2.823112
6,ID723,2.829149
7,ID724,2.788104
8,ID725,2.765896
9,ID727,2.751374


In [47]:
best = pd.read_csv('submission20180107_102857-0.03974.csv', names=['best'])
best['pred'] = pd.DataFrame(result)[0].values
best

,best,pred
ID716,2.671755,2.686064
ID717,2.702322,2.711856
ID719,2.924776,2.905117
ID720,3.000923,2.966445
ID721,2.788795,2.798389
ID722,2.851759,2.823349
ID723,2.831660,2.831276
ID724,2.852925,2.788803
ID725,2.808842,2.767407
ID727,2.823874,2.765588


In [44]:
a = pd.read_csv('submission20180106_211141-0.03977.csv', names=['res'])
b = pd.read_csv('submission20180107_102857-0.03974.csv', names=['res1'])
a['res1'] = sub['res'].values
a['res2'] = b['res1']
a['sub'] = a.mean(axis=1)
a

,res,res1,res2,sub
ID716,2.670762,2.686064,2.671755,2.676194
ID717,2.702805,2.711856,2.702322,2.705661
ID719,2.924748,2.905117,2.924776,2.918214
ID720,3.000912,2.966445,3.000923,2.989427
ID721,2.790809,2.798389,2.788795,2.792664
ID722,2.854324,2.823349,2.851759,2.843144
ID723,2.833468,2.831276,2.831660,2.832135
ID724,2.854338,2.788803,2.852925,2.832022
ID725,2.807870,2.767407,2.808842,2.794707
ID727,2.825297,2.765588,2.823874,2.804919


In [45]:
sub = pd.read_csv('data/测试A-答案模板.csv', names=['ID'])
sub['res'] = a['sub'].values
sub

,ID,res
0,ID716,2.676194
1,ID717,2.705661
2,ID719,2.918214
3,ID720,2.989427
4,ID721,2.792664
5,ID722,2.843144
6,ID723,2.832135
7,ID724,2.832022
8,ID725,2.794707
9,ID727,2.804919


In [51]:
sub.to_csv('submission{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)

线下

In [ ]:
#5折交叉验证
x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2, random_state=1)
x_train.reset_index(inplace=True)
x_train.drop(['index'], axis=1, inplace=True)

y_train.reset_index(inplace=True)
y_train.drop(['index'], axis=1, inplace=True)

y_valid.reset_index(inplace=True)
y_valid.drop(['index'], axis=1, inplace=True)

In [57]:
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, max_depth=2, subsample=0.8, learning_rate=0.01, random_state=1, max_features=0.3)
modle0  = xgb.XGBRegressor(learning_rate=0.01, max_depth=3, colsample_bytree=0.3, subsample=0.8, seed=0, n_estimators=1100)
modle1  = xgb.XGBRegressor(learning_rate=0.01, max_depth=4, colsample_bytree=0.3, subsample=0.8, seed=0, n_estimators=1200)
sv = svm.SVR()
tree1 = tree.ExtraTreeRegressor()
xgb1 = xgb.XGBRegressor()
lgb1 = lgb.LGBMRegressor()
et = linear_model.LinearRegression()

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3, 
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=250, 
                         seed=1)

base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['lgb', clf1],]

folds = list(KFold(len(x_train), n_folds=5, random_state=0))
S_train = np.zeros((x_train.shape[0], len(base_model)))
S_test = np.zeros((x_valid.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((x_valid.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = x_train.ix[train_idx, :]
        X_valid = x_train.ix[test_idx, :]
        Y = y_train.ix[train_idx, :]
        clf.fit(X_train, Y['Y'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(x_valid)       
    S_test[:, index] = S_test_i.mean(1)
    
linreg = svm.SVR()
linreg1 = linear_model.LinearRegression()
linreg1.fit(S_train, y_train)

result = linreg1.predict(S_test)
evalerror(pd.DataFrame(result)[0],  y_valid['Y'])

the model 0
the model 1
the model 2
the model 3


0.025541442893531906